## การฝังข้อมูล

ในตัวอย่างก่อนหน้านี้ เราได้ทำงานกับเวกเตอร์แบบถุงคำ (bag-of-words) ที่มีมิติสูงและมีความยาวเท่ากับ `vocab_size` และเราได้แปลงเวกเตอร์ที่แสดงตำแหน่งในมิติต่ำให้เป็นการแสดงผลแบบ one-hot ที่มีความเบาบาง การแสดงผลแบบ one-hot นี้ไม่ประหยัดหน่วยความจำ นอกจากนี้ แต่ละคำยังถูกพิจารณาแยกจากกัน ทำให้เวกเตอร์แบบ one-hot ไม่สามารถแสดงความคล้ายคลึงทางความหมายระหว่างคำได้

ในหน่วยนี้ เราจะสำรวจชุดข้อมูล **News AG** ต่อไป เพื่อเริ่มต้น เรามาโหลดข้อมูลและนำคำจำกัดความบางส่วนจากหน่วยก่อนหน้ามาใช้กัน


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### อะไรคือ Embedding?

แนวคิดของ **embedding** คือการแทนคำด้วยเวกเตอร์ที่มีมิติที่ต่ำกว่าและหนาแน่น ซึ่งสะท้อนถึงความหมายเชิงความหมายของคำ เราจะพูดถึงวิธีการสร้าง word embeddings ที่มีความหมายในภายหลัง แต่ตอนนี้ให้คิดว่า embedding เป็นวิธีลดมิติของเวกเตอร์คำ

ดังนั้น embedding layer จะรับคำเป็น input และสร้าง output เป็นเวกเตอร์ที่มีขนาด `embedding_size` ในแง่หนึ่งมันคล้ายกับ `Dense` layer แต่แทนที่จะรับเวกเตอร์แบบ one-hot encoded เป็น input มันสามารถรับหมายเลขคำได้

เมื่อใช้ embedding layer เป็นเลเยอร์แรกในเครือข่ายของเรา เราสามารถเปลี่ยนจาก bag-of-words ไปเป็นโมเดล **embedding bag** โดยที่เราจะแปลงแต่ละคำในข้อความของเราให้เป็น embedding ที่สอดคล้องกัน และคำนวณฟังก์ชันรวมบางอย่างจาก embeddings เหล่านั้น เช่น `sum`, `average` หรือ `max`

![ภาพแสดงตัวอย่าง embedding classifier สำหรับคำในลำดับห้าคำ](../../../../../translated_images/th/embedding-classifier-example.b77f021a7ee67eee.webp)

เครือข่ายประสาทสำหรับการจำแนกของเราประกอบด้วยเลเยอร์ดังต่อไปนี้:

* เลเยอร์ `TextVectorization` ซึ่งรับสตริงเป็น input และสร้างเทนเซอร์ของหมายเลขโทเค็น เราจะกำหนดขนาดคำศัพท์ `vocab_size` ที่เหมาะสม และละเว้นคำที่ใช้น้อยกว่า รูปร่างของ input จะเป็น 1 และรูปร่างของ output จะเป็น $n$ เนื่องจากเราจะได้ $n$ โทเค็นเป็นผลลัพธ์ โดยแต่ละโทเค็นจะมีหมายเลขตั้งแต่ 0 ถึง `vocab_size`
* เลเยอร์ `Embedding` ซึ่งรับ $n$ หมายเลข และลดแต่ละหมายเลขลงเป็นเวกเตอร์หนาแน่นที่มีความยาวที่กำหนด (100 ในตัวอย่างของเรา) ดังนั้นเทนเซอร์ input ที่มีรูปร่าง $n$ จะถูกแปลงเป็นเทนเซอร์ $n\times 100$
* เลเยอร์การรวม (Aggregation layer) ซึ่งคำนวณค่าเฉลี่ยของเทนเซอร์นี้ตามแกนแรก กล่าวคือมันจะคำนวณค่าเฉลี่ยของเทนเซอร์ input $n$ ทั้งหมดที่สอดคล้องกับคำต่าง ๆ เพื่อสร้างเลเยอร์นี้ เราจะใช้เลเยอร์ `Lambda` และส่งฟังก์ชันที่คำนวณค่าเฉลี่ยเข้าไป รูปร่างของ output จะเป็น 100 และมันจะเป็นตัวแทนเชิงตัวเลขของลำดับ input ทั้งหมด
* ตัวจำแนกเชิงเส้น `Dense` สุดท้าย


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


ใน `summary` ที่แสดงผล ในคอลัมน์ **output shape** มิติแรกของเทนเซอร์ `None` หมายถึงขนาดของมินิแบตช์ และมิติที่สองหมายถึงความยาวของลำดับโทเค็น ลำดับโทเค็นทั้งหมดในมินิแบตช์มีความยาวที่แตกต่างกัน เราจะพูดถึงวิธีจัดการกับสิ่งนี้ในส่วนถัดไป

ตอนนี้มาเริ่มฝึกเครือข่ายกัน:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **หมายเหตุ** เรากำลังสร้างเวกเตอไรเซอร์โดยใช้ข้อมูลเพียงบางส่วนเท่านั้น การทำเช่นนี้เพื่อเพิ่มความเร็วในกระบวนการ และอาจส่งผลให้บางโทเค็นจากข้อความของเราไม่ปรากฏในคลังคำศัพท์ ในกรณีนี้ โทเค็นเหล่านั้นจะถูกละเว้น ซึ่งอาจทำให้ความแม่นยำลดลงเล็กน้อย อย่างไรก็ตาม ในชีวิตจริง ข้อความบางส่วนมักให้การประมาณคลังคำศัพท์ที่ดี


### การจัดการกับขนาดลำดับตัวแปร

มาทำความเข้าใจว่าการฝึกอบรมในรูปแบบมินิแบตช์เกิดขึ้นได้อย่างไร ในตัวอย่างข้างต้น เทนเซอร์อินพุตมีมิติเท่ากับ 1 และเราใช้มินิแบตช์ที่มีความยาว 128 ดังนั้นขนาดจริงของเทนเซอร์คือ $128 \times 1$ อย่างไรก็ตาม จำนวนโทเค็นในแต่ละประโยคจะแตกต่างกันออกไป หากเราใช้เลเยอร์ `TextVectorization` กับอินพุตเดียว จำนวนโทเค็นที่ได้จะไม่เท่ากัน ขึ้นอยู่กับวิธีการที่ข้อความถูกแปลงเป็นโทเค็น:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


อย่างไรก็ตาม เมื่อเราใช้เวคเตอไรเซอร์กับลำดับหลายชุด มันจะต้องสร้างเทนเซอร์ที่มีรูปทรงสี่เหลี่ยม ดังนั้นมันจึงเติมองค์ประกอบที่ไม่ได้ใช้งานด้วยโทเค็น PAD (ซึ่งในกรณีของเราคือศูนย์):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

ที่นี่เราสามารถเห็นการฝังข้อมูล:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

## การฝังความหมาย: Word2Vec

ในตัวอย่างก่อนหน้านี้ เลเยอร์การฝัง (embedding layer) ได้เรียนรู้การแปลงคำให้เป็นเวกเตอร์ แต่เวกเตอร์เหล่านั้นยังไม่มีความหมายในเชิงความหมาย (semantic meaning) จะดีกว่าหากเราสามารถเรียนรู้การแปลงคำให้เป็นเวกเตอร์ที่คำที่มีความหมายคล้ายกันหรือคำพ้องความหมายมีเวกเตอร์ที่อยู่ใกล้กันในแง่ของระยะทางเวกเตอร์บางประเภท (เช่น ระยะทางแบบยุคลิด)

เพื่อทำเช่นนั้น เราจำเป็นต้องฝึกโมเดลการฝังของเราล่วงหน้าด้วยชุดข้อความขนาดใหญ่ โดยใช้เทคนิคอย่าง [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) ซึ่งอิงตามสถาปัตยกรรมหลักสองแบบที่ใช้ในการสร้างการแสดงผลแบบกระจายของคำ:

 - **Continuous bag-of-words** (CBoW) ซึ่งเราฝึกโมเดลให้ทำนายคำจากบริบทโดยรอบ โดยให้ ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ เป้าหมายของโมเดลคือการทำนาย $W_0$ จาก $(W_{-2},W_{-1},W_1,W_2)$
 - **Continuous skip-gram** ตรงข้ามกับ CBoW โมเดลจะใช้บริบทของคำรอบข้างเพื่อทำนายคำปัจจุบัน

CBoW ทำงานได้เร็วกว่า ในขณะที่ skip-gram แม้จะช้ากว่า แต่สามารถแสดงคำที่พบได้น้อยได้ดีกว่า

![ภาพแสดงอัลกอริธึม CBoW และ Skip-Gram สำหรับแปลงคำเป็นเวกเตอร์](../../../../../translated_images/th/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

เพื่อทดลองใช้การฝัง Word2Vec ที่ฝึกไว้ล่วงหน้าด้วยชุดข้อมูล Google News เราสามารถใช้ไลบรารี **gensim** ด้านล่างนี้เป็นตัวอย่างการค้นหาคำที่คล้ายกับ 'neural' มากที่สุด

> **Note:** เมื่อคุณสร้างเวกเตอร์คำครั้งแรก การดาวน์โหลดอาจใช้เวลาสักครู่!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


เรายังสามารถดึงเวกเตอร์ฝังตัวจากคำมาใช้ในการฝึกโมเดลการจำแนกได้ เวกเตอร์ฝังตัวมี 300 องค์ประกอบ แต่ที่นี่เราจะแสดงเฉพาะ 20 องค์ประกอบแรกของเวกเตอร์เพื่อความชัดเจน:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

ตัวอย่างข้างต้นใช้เวทมนตร์ภายในของ GenSym บางอย่าง แต่ตรรกะพื้นฐานนั้นค่อนข้างง่าย สิ่งที่น่าสนใจเกี่ยวกับการฝังตัวคือคุณสามารถดำเนินการเวกเตอร์ปกติบนเวกเตอร์การฝังตัว และนั่นจะสะท้อนถึงการดำเนินการบน **ความหมาย** ของคำ ตัวอย่างข้างต้นสามารถแสดงออกในแง่ของการดำเนินการเวกเตอร์: เราคำนวณเวกเตอร์ที่สอดคล้องกับ **KING-MAN+WOMAN** (การดำเนินการ `+` และ `-` ถูกดำเนินการบนตัวแทนเวกเตอร์ของคำที่เกี่ยวข้อง) และจากนั้นค้นหาคำที่ใกล้เคียงที่สุดในพจนานุกรมกับเวกเตอร์นั้น:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: เราต้องเพิ่มค่าสัมประสิทธิ์เล็กน้อยให้กับเวกเตอร์ *man* และ *woman* - ลองลบออกดูเพื่อดูว่าจะเกิดอะไรขึ้น

ในการหาเวกเตอร์ที่ใกล้ที่สุด เราใช้เครื่องมือของ TensorFlow เพื่อคำนวณเวกเตอร์ของระยะห่างระหว่างเวกเตอร์ของเราและเวกเตอร์ทั้งหมดในคำศัพท์ จากนั้นจึงหาอินเด็กซ์ของคำที่มีค่าน้อยที่สุดโดยใช้ `argmin`


ในขณะที่ Word2Vec ดูเหมือนจะเป็นวิธีที่ยอดเยี่ยมในการแสดงความหมายของคำ แต่ก็มีข้อเสียหลายประการ รวมถึง:

* ทั้งโมเดล CBoW และ skip-gram เป็น **predictive embeddings** ซึ่งพิจารณาเพียงบริบทในพื้นที่เท่านั้น Word2Vec ไม่ได้ใช้ประโยชน์จากบริบทในภาพรวม
* Word2Vec ไม่ได้คำนึงถึง **morphology** ของคำ เช่น ความหมายของคำที่อาจขึ้นอยู่กับส่วนต่าง ๆ ของคำ เช่น รากศัพท์  

**FastText** พยายามแก้ไขข้อจำกัดในข้อที่สอง โดยพัฒนาต่อยอดจาก Word2Vec ด้วยการเรียนรู้การแสดงผลแบบเวกเตอร์สำหรับแต่ละคำและ n-grams ของตัวอักษรที่พบในแต่ละคำ จากนั้นค่าของการแสดงผลเหล่านี้จะถูกเฉลี่ยเป็นเวกเตอร์เดียวในแต่ละขั้นตอนการฝึก แม้ว่าวิธีนี้จะเพิ่มการคำนวณเพิ่มเติมในขั้นตอนการฝึกเบื้องต้น แต่มันช่วยให้ word embeddings สามารถเข้ารหัสข้อมูลในระดับย่อยของคำได้

อีกวิธีหนึ่งคือ **GloVe** ซึ่งใช้แนวทางที่แตกต่างในการสร้าง word embeddings โดยอิงจากการแยกตัวประกอบของเมทริกซ์ word-context ขั้นแรกจะสร้างเมทริกซ์ขนาดใหญ่ที่นับจำนวนการปรากฏของคำในบริบทต่าง ๆ จากนั้นพยายามแสดงเมทริกซ์นี้ในมิติที่ต่ำกว่าในลักษณะที่ลดการสูญเสียจากการสร้างใหม่ให้น้อยที่สุด

ไลบรารี gensim รองรับ word embeddings เหล่านี้ และคุณสามารถทดลองใช้งานได้โดยการเปลี่ยนโค้ดการโหลดโมเดลด้านบน


## การใช้ pretrained embeddings ใน Keras

เราสามารถปรับตัวอย่างข้างต้นเพื่อเติมข้อมูลในเมทริกซ์ของ embedding layer ด้วย semantic embeddings เช่น Word2Vec ได้ คำศัพท์ของ pretrained embedding และ text corpus อาจไม่ตรงกัน ดังนั้นเราจำเป็นต้องเลือกหนึ่งในสองตัวเลือก ที่นี่เราจะสำรวจสองตัวเลือกที่เป็นไปได้: การใช้คำศัพท์จาก tokenizer และการใช้คำศัพท์จาก Word2Vec embeddings

### การใช้คำศัพท์จาก tokenizer

เมื่อใช้คำศัพท์จาก tokenizer บางคำในคำศัพท์จะมี Word2Vec embeddings ที่สอดคล้องกัน และบางคำจะไม่มี โดยที่ขนาดของคำศัพท์ของเราคือ `vocab_size` และความยาวของเวกเตอร์ Word2Vec embedding คือ `embed_size` embedding layer จะถูกแทนด้วยเมทริกซ์น้ำหนักที่มีรูปร่าง `vocab_size`$\times$`embed_size` เราจะเติมข้อมูลในเมทริกซ์นี้โดยการไล่ผ่านคำศัพท์:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


สำหรับคำที่ไม่มีอยู่ในคลังคำศัพท์ของ Word2Vec เราสามารถเลือกที่จะปล่อยให้เป็นค่าเป็นศูนย์ หรือสร้างเวกเตอร์แบบสุ่มขึ้นมาแทน

ตอนนี้เราสามารถกำหนดชั้น embedding ที่มีน้ำหนักที่ถูกฝึกไว้ล่วงหน้าได้:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **หมายเหตุ**: สังเกตว่าเราได้ตั้งค่า `trainable=False` เมื่อสร้าง `Embedding` ซึ่งหมายความว่าเราไม่ได้ฝึกฝนเลเยอร์ Embedding ใหม่ การทำเช่นนี้อาจทำให้ความแม่นยำลดลงเล็กน้อย แต่จะช่วยให้การฝึกฝนเร็วขึ้น

### การใช้คำศัพท์ของ embedding

ปัญหาหนึ่งของวิธีการก่อนหน้านี้คือคำศัพท์ที่ใช้ใน TextVectorization และ Embedding นั้นแตกต่างกัน เพื่อแก้ปัญหานี้ เราสามารถใช้หนึ่งในวิธีการต่อไปนี้:
* ฝึกฝนโมเดล Word2Vec ใหม่โดยใช้คำศัพท์ของเรา
* โหลดชุดข้อมูลของเราด้วยคำศัพท์จากโมเดล Word2Vec ที่ผ่านการฝึกฝนมาแล้ว คำศัพท์ที่ใช้ในการโหลดชุดข้อมูลสามารถระบุได้ในระหว่างการโหลด

วิธีที่สองดูเหมือนจะง่ายกว่า ดังนั้นเรามาลองทำกัน ก่อนอื่นเราจะสร้างเลเยอร์ `TextVectorization` โดยใช้คำศัพท์ที่ระบุ ซึ่งนำมาจาก Word2Vec embeddings:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

ไลบรารี gensim word embeddings มีฟังก์ชันที่สะดวกชื่อ `get_keras_embeddings` ซึ่งจะสร้างเลเยอร์ Keras embeddings ที่สอดคล้องกันให้คุณโดยอัตโนมัติ


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


หนึ่งในเหตุผลที่เราไม่เห็นความแม่นยำที่สูงขึ้นเป็นเพราะคำบางคำจากชุดข้อมูลของเราหายไปในคลังคำศัพท์ GloVe ที่ผ่านการฝึกมาแล้ว และดังนั้นคำเหล่านั้นจึงถูกมองข้ามไป เพื่อแก้ปัญหานี้ เราสามารถฝึกฝนการฝังตัวคำของเราเองโดยอ้างอิงจากชุดข้อมูลของเรา


## การฝังบริบท

ข้อจำกัดสำคัญของการฝังคำแบบดั้งเดิมที่ผ่านการฝึกอบรมล่วงหน้า เช่น Word2Vec คือ แม้ว่ามันจะสามารถจับความหมายบางส่วนของคำได้ แต่ก็ไม่สามารถแยกแยะความหมายที่แตกต่างกันได้ ซึ่งอาจทำให้เกิดปัญหาในโมเดลที่ใช้งานต่อเนื่อง

ตัวอย่างเช่น คำว่า 'play' มีความหมายต่างกันในสองประโยคนี้:
- ฉันไปดู **ละคร** ที่โรงละคร
- จอห์นอยาก **เล่น** กับเพื่อนของเขา

การฝังคำที่ผ่านการฝึกอบรมล่วงหน้าที่เราได้พูดถึงนั้นจะเป็นตัวแทนของทั้งสองความหมายของคำว่า 'play' ในการฝังเดียวกัน เพื่อแก้ไขข้อจำกัดนี้ เราจำเป็นต้องสร้างการฝังคำที่อิงตาม **โมเดลภาษา** ซึ่งได้รับการฝึกอบรมจากชุดข้อความขนาดใหญ่ และ *เข้าใจ* ว่าคำสามารถนำมารวมกันในบริบทที่แตกต่างกันได้อย่างไร การพูดคุยเกี่ยวกับการฝังบริบทอยู่นอกเหนือขอบเขตของบทเรียนนี้ แต่เราจะกลับมาพูดถึงเรื่องนี้อีกครั้งเมื่อพูดถึงโมเดลภาษาในหน่วยถัดไป



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
